In [1]:
import rospy
import gym, ray
import numpy as np
import os
import torch.nn as nn

from ray import tune
from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from scouting_gym.tasks.scouting_discrete_task import ScoutingDiscreteTask

from ray.tune import grid_search
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.visionnet import VisionNetwork
import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer
from ray.rllib.models.tf.misc import normc_initializer
from ray.tune.logger import pretty_print
from matplotlib import pyplot as plt
import tensorflow as tf

Instructions for updating:
non-resource variables are not supported in the long term
None


In [2]:
env = gym.make('Scouting-v0')

print(env.observation_space)

[ERROR] [1610532865.454433, 0.000000]: NOT Initialising Simulation Physics Parameters
[WARN] [1610532865.459179, 0.000000]: Start Init ControllersConnection
[WARN] [1610532865.460429, 0.000000]: END Init ControllersConnection


Box(0.0, 10.0, (300,), float32)


In [3]:
env.reset()
for _ in range(10):
    obs, reward, done, _ = env.step(1)
print(obs.min())
print(obs.max())

1.1700057
6.826673


In [4]:
class CustomModel(TFModelV2):
    """Example of a keras custom model that just delegates to an fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        super(CustomModel, self).__init__(obs_space, action_space, num_outputs,
                                          model_config, name)
        self.model = FullyConnectedNetwork(obs_space, action_space,
                                           num_outputs, model_config, name)
        self.register_variables(self.model.variables())

    def forward(self, input_dict, state, seq_lens):
        return self.model.forward(input_dict, state, seq_lens)

    def value_function(self):
        return self.model.value_function()

class MyKerasModel(TFModelV2):
    """Custom model for policy gradient algorithms."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        super(MyKerasModel, self).__init__(obs_space, action_space,
                                           num_outputs, model_config, name)
        self.inputs = tf.keras.layers.Input(
            shape=obs_space.shape, name="observations")

        layer_dense_1 = tf.keras.layers.Dense(
            256,
            name="Dense1",
            activation=tf.nn.relu,
            kernel_initializer=normc_initializer(1.0))(self.inputs)

        layer_dense_2 = tf.keras.layers.Dense(
            128,
            name="Dense2",
            activation=tf.nn.relu,
            kernel_initializer=normc_initializer(1.0))(layer_dense_1)
        layer_out = tf.keras.layers.Dense(
            num_outputs,
            name="my_out",
            activation=None,
            kernel_initializer=normc_initializer(0.01))(layer_dense_2)
        value_out = tf.keras.layers.Dense(
            1,
            name="value_out",
            activation=None,
            kernel_initializer=normc_initializer(0.01))(layer_dense_2)
        self.base_model = tf.keras.Model(self.inputs, [layer_out, value_out])
        self.register_variables(self.base_model.variables)

    def forward(self, input_dict, state, seq_lens):
        model_out, self._value_out = self.base_model(input_dict["obs"])
        return model_out, state

    def value_function(self):
        return tf.reshape(self._value_out, [-1])

    def metrics(self):
        return {"foo": tf.constant(42.0)}

In [5]:
ray.init()

ModelCatalog.register_custom_model(
    "my_model", CustomModel)

config = {
    "env": ScoutingDiscreteTask,  # or "corridor" if registered above
    "env_config": {
        "corridor_length": 5,
    },
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.

    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
    "model": {
        "custom_model": "my_model",
    },
    "num_workers": 1,  # parallelism
    "vf_share_layers": False,
}



stop = {
    "training_iteration": 500,
    "timesteps_total": 1000000
}

2021-01-13 11:14:29,049	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


In [6]:
trainer = PPOTrainer(config=config)
policy = trainer.get_policy()

print(policy.model.model.base_model.summary())

2021-01-13 11:14:30,517	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-01-13 11:14:30,518	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=55502) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=55502) Instructions for updating:
(pid=55502) non-resource variables are not supported in the long term
(pid=55502) [ERROR] [1610532872.917387, 0.000000]: NOT Initialising Simulation Physics Parameters
(pid=55502) [WARN] [1610532872.922286, 0.000000]: Start Init ControllersConnection
(pid=55502) [WARN] [1610532872.923339, 0.000000]: END Init ControllersConnection
2021-01-13 11:14:37,523	WARNING util.py:43 -- Install gputil for GPU system monitoring.


(pid=55502) None
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observations (InputLayer)       [(None, 300)]        0                                            
__________________________________________________________________________________________________
fc_1 (Dense)                    (None, 256)          77056       observations[0][0]               
__________________________________________________________________________________________________
fc_value_1 (Dense)              (None, 256)          77056       observations[0][0]               
__________________________________________________________________________________________________
fc_2 (Dense)                    (None, 256)          65792       fc_1[0][0]                       
_____________________________________________________________________________

In [ ]:
results = []
for i in range(100):
    result = trainer.train()
    results.append(result)

    if i % 5 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)

    if i % 20 == 0:
        print(pretty_print(result))

(pid=55502) 2021-01-13 11:14:38,759	WARNING deprecation.py:29 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=55502) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/ray/rllib/policy/tf_policy.py:850: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=55502) Instructions for updating:
(pid=55502) Prefer Variable.assign which has equivalent behavior in 2.X.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
checkpoint saved at /home/dschori/ray_results/PPO_ScoutingDiscreteTask_2021-01-13_11-14-30ly4u9ku2/checkpoint_1/checkpoint-1
custom_metrics: {}
date: 2021-01-13_11-16-37
done: false
episode_len_mean: 199.85
episode_reward_max: -126.08000000000013
episode_reward_mean: -161.6225
episode_reward_min: -230.37000000000003
episodes_this_iter: 20
episodes_total: 20
experiment_id: 08552b7c7927455dbeb82ed613a10ca6
hostname: workstation
info:
  learner:
    default_policy:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 4.999999873689376e-05
      entropy: 1.0784943103790283
      entropy_coeff: 0.0
      kl: 0.020526276901364326
      model: {}
      policy_loss: -0.0436776764690876
      total_loss: 7196.58447265625
      vf_explained_var: 0.05757300928235054
      vf_loss: 7196.625
  num_steps_sampled: 4000
  num_steps_trained: 4000
iterations_since_restore: 1
node_ip: 192.168.178.60
num_healthy_worke